In [ ]:
# Execute this first!
#
# Our new data set has a lot of columns, so we want the
# notebook to display more... 30, say.

options(repr.matrix.max.cols=30)

The Clemency Initiative - In-Class/Take-Home Drill
-----------------------

We have been studying [the web page of Obama's commutations](https://www.justice.gov/pardon/obama-commutations). Our ultimate goal is to create a data frame that would let us operate on the data more conveniently. So how do we take the free text of [the commutations web page](https://www.justice.gov/pardon/obama-commutations) and systematically fill in a more structured data set? As we go through this process, it's a good idea to also consult the [Clemency Initiative](https://www.justice.gov/pardon/clemency-initiative) for federal inmates, its goals and what it accomplished under Obama. 

Here are some post-mortems of the project.

> http://thehill.com/homenews/administration/315107-obama-issues-final-round-of-sentence-commutations
<br><br>
https://www.washingtonpost.com/world/national-security/obama-grants-final-330-commutations-to-nonviolent-drug-offenders/2017/01/19/41506468-de5d-11e6-918c-99ede3c8cafa_story.html?utm_term=.5c57a8437c9e
<br><br>
https://www.justice.gov/pardon/clemency-statistics

We start by loading the libraries we'll need (mostly offerings from Hadley Wickham) and then read in the latest version of the data produced by the "Meeting 7" notebook. Oh and since we are reading from a CSV, R will automatically take our lovely dates and read them as strings. If you download newcomms2.csv from CourseWorks and open it in a spreadsheet, you'll see that the date_1 column has entries like "2008-03-05", for example.

To turn them back into date objects we use ymd() from "lubridate" (year-month-date, to match our date_1 format). That's the third group of commands in the block below.

In [ ]:
library(rvest)
library(dplyr)
library(lubridate)
library(reshape2)
library(stringr)

newcommutations = read.csv("newcomms2.csv",as.is=TRUE)

newcommutations = mutate(newcommutations,date=ymd(date))
newcommutations = mutate(newcommutations,date_2=ymd(date_2))
newcommutations = mutate(newcommutations,date_3=ymd(date_3))

The code below creates a random selection of rows using the command sample_n(). This is often a good strategy because it lets you spot check your data, rather than focusing on just the head() or the tail() of the data set. 

Execute the cell below and look at the different cells, especially the last few that were derived from free text fields like the district_date or the offense. 

In [ ]:
sample_n(newcommutations,25)

Have a look at the original first offenses and see what dates they were committed in. How would you summarize this?  Maybe just a simple histogram of date?

In [ ]:
# put your code here


In [ ]:
... or maybe use years() to extract the year of the offense and then a barplot?

In [ ]:
# put your code here


In the previous session, we saw how we might use regular expressons to extract facts about the free text elements of our web page. First we looked at the states where people were convicted. This means we need a way to look at the "district_date" field for each inmate and extract the state name. 

The easiest regular expression is just a set of literals. So the expression "New Jersey" is asking us to match the character string "New Jersey" exactly - so an "N" then an "e" then a "w" and so on. 

We use the "stringr" command str_detect() create a boolean (TRUE/FALSE) vector that is TRUE if "New Jersey" is in the "district_date" field and FALSE otherwise. The command filter() can then be used with this boolean vector to select just the rows with "New Jersey" in the "district_date" field.

Here are just the commutations from New Jersey.

In [ ]:
filter(newcommutations,str_detect(district_date,"New Jersey"))

If we want commutations from either New Jersey or Massachusetts, we could join the two words in our regular expression into a pair of alternatives, separating them with a vertical bar "|". The expression is "New Jersey|Massachusetts". 

In [ ]:
filter(newcommutations,str_detect(district_date,"New Jersey|Massachusetts"))

The expression "New Jersey|Massachusetts" can be elaborated a little. We probably don't need this for state names, but if we were matching colors like "red" and "green" in text, we have to be careful because "green" would match "greenwich" and "red" might match "hundred". So if we want to match words, we could surround them with a "character class" that represents a word boundary. That's "\\b". We mentioned that is also special character classes like "\\w" representing any word character. This would include "a-zA-Z" for example.

So, intead of "New Jersey|Massachusetts", we might use "\\bNew Jersey\\b|\\bMassachusetts\\b" which suggests looking for word boundaries around "New Jersey" or "Massachusetts". You can see the difference by looking at [regexper.com](https://regexper.com/#%5CbNew%20Jersey%5Cb%7C%5CbMassachusetts%5Cb). 

One last thing, because R uses the backslash for an escape character, we have to double all backslashes. So whatever works in regexper.com, we need to double the backslashes for entering them into R. Here's the "or" for our two states.

In [ ]:
filter(newcommutations,str_detect(district_date,"\\bNew Jersey\\b|\\bMassachusetts\\b"))

OK, so if we want to look for any state, we don't want to have to keep typing state names. For this purpose,we can use a built-in data set in R called "state.name." It is, well, a vector of strings, the names of the 50 states. 

We load this and a number of other data sets about the 50 states using the data() command. It is similar to library() but for the data sets that come with R. You can get a complete list using the command without any arguments. (To close the window list, click the "x" in the upper right corner below.)

In [ ]:
data()

This is a pretty esoteric collection of data. It represents a mix of data that are important for data analysis as well for statistics education. There are "classic" data sets that instructors use in classes, and the most popular ones drift into languages like R.

Let's just look at the data on the states. While we have data sets like "states.abb" for abbreviations, we also have state names.

In [ ]:
data(state)
state.name

In [ ]:
state.abb

With a little work, we'd also see that there are some district that aren't strictly in states. There's Puerto Rico, Guam and DC, for example. We can add these to our states.

In [ ]:
state.name = c(state.name,"Puerto Rico","Guam","District of Columbia","U.S. Army Court Martial")

To be safe, we are going to make all our matches with lower-case versions of the state names. We then needed to craft a regular expression that was essential a series of "or" conditions -- arkansas or alabama or alaska... We fashioned this from "state.name" using the command in "stringr" called str_c(). It takes vectors of strings and glues them together. 

Here we stuck the state names between two "\\\b" character classes and separated each state with a vertical bar. Again, the "\\\b" means a word boundary (like a space or some punctuation) and the vertical bar means "or". 

The expression below can be visualized using [regexper.com](https://regexper.com/#%5Cbalabama%5Cb%7C%5Cbalaska%5Cb%7C%5Cbarizona%5Cb%7C%5Cbarkansas%5Cb%7C%5Cbcalifornia%5Cb%7C%5Cbcolorado%5Cb%7C%5Cbconnecticut%5Cb%7C%5Cbdelaware%5Cb%7C%5Cbflorida%5Cb%7C%5Cbgeorgia%5Cb%7C%5Cbhawaii%5Cb%7C%5Cbidaho%5Cb%7C%5Cbillinois%5Cb%7C%5Cbindiana%5Cb%7C%5Cbiowa%5Cb%7C%5Cbkansas%5Cb%7C%5Cbkentucky%5Cb%7C%5Cblouisiana%5Cb%7C%5Cbmaine%5Cb%7C%5Cbmaryland%5Cb%7C%5Cbmassachusetts%5Cb%7C%5Cbmichigan%5Cb%7C%5Cbminnesota%5Cb%7C%5Cbmississippi%5Cb%7C%5Cbmissouri%5Cb%7C%5Cbmontana%5Cb%7C%5Cbnebraska%5Cb%7C%5Cbnevada%5Cb%7C%5Cbnew%20hampshire%5Cb%7C%5Cbnew%20jersey%5Cb%7C%5Cbnew%20mexico%5Cb%7C%5Cbnew%20york%5Cb%7C%5Cbnorth%20carolina%5Cb%7C%5Cbnorth%20dakota%5Cb%7C%5Cbohio%5Cb%7C%5Cboklahoma%5Cb%7C%5Cboregon%5Cb%7C%5Cbpennsylvania%5Cb%7C%5Cbrhode%20island%5Cb%7C%5Cbsouth%20carolina%5Cb%7C%5Cbsouth%20dakota%5Cb%7C%5Cbtennessee%5Cb%7C%5Cbtexas%5Cb%7C%5Cbutah%5Cb%7C%5Cbvermont%5Cb%7C%5Cbvirginia%5Cb%7C%5Cbwashington%5Cb%7C%5Cbwest%20virginia%5Cb%7C%5Cbwisconsin%5Cb%7C%5Cbwyoming%5Cb).

In [ ]:
reg = str_c("\\b",tolower(state.name),"\\b",collapse="|")
print(reg)

We then pass this to another "stringr" function called str_extract(). It will extract the data that matches our plattern. Notice that we again take the "district_date" and use tolower()  to make lowercase comparisons. 

In this case, a match means we have found one of the states' names. Here are all 945 state names found in the "district_date" field. 

In [ ]:
str_extract(tolower(newcommutations$district_date),reg)

This is how we made a new column in our data set consisting of state names (that plus correcting one spelling error, "Wisconson"). We called the variables "state," "state_2" and "state_3" corresponding to the first, second and third districts associated with an inmates' prison sentence. I've commented out these lines of code since your data set already has these columns.

Just to give you a chance to check things out, the last line of code here creates a sample of the commutations again.

In [ ]:
# newcommutations = mutate(newcommutations,state=str_extract(tolower(district_date),reg))
# newcommutations = mutate(newcommutations,state_2=str_extract(tolower(district_date_2),reg))
# newcommutations = mutate(newcommutations,state_3=str_extract(tolower(district_date_3),reg))

sample_n(newcommutations,10)

In [ ]:
sort(table(newcommutations$state))

Next let's look at sentencing. We might try looking at sections of the U.S. code mentioned in the "offense" fields. We can search for "§", the "section sign" and created a regular expression with just this character. how many of the inmates in our data set have this character in their offense?

In [ ]:
# put your code here


In addition, we can look at the words in the ofense listing. Here is a test for heroin or LSD offenses. 

In [ ]:
druglist = "\\blsd\\b|\\bheroin\\b"
drugs = filter(newcommutations,!(str_detect(tolower(offense),druglist)))
sample_n(drugs,5)

Here is a more elaborate list of drugs. This time, we use the "!" to turn our TRUEs into FALSEs and look for offenses that don't include one of the drug names. Let's try that out and read the sentences. 

Also, have a look at the regular expression on [regexper.com](https://regexper.com/#%5Cbcrack%5Cb%7C%5Cblsd%5Cb%7C%5Cbphencyclidine%5Cb%7C%5Cbnarcotic%5Cb%7C%5Cbdrug%5Cb%7C%5Cbcontrolled%20substance%5Cb%7C%5Cbheroin%5Cb%7C%5Cbcocaine%5Cb%7C%5Cbmari%28j%7Ch%29uana%5Cb%7C%5Cbmethamphetamine%5Cb)

In [ ]:
druglist = "\\bphencyclidine\\b|\\bnarcotic\\b|\\bdrug\\b|\\bcontrolled substance\\b|\\bheroin\\b|\\bcocaine\\b|\\bmari(j|h)uana\\b|\\bmethamphetamine\\b"
nodrugs = filter(newcommutations,!(str_detect(tolower(offense),druglist)))
nodrugs

I came up with the list of drugs by looking at samples of the "left overs" like in the "nodrugs" data frame. This list could clearly be added to. Kenneth Isaacs, for example, was committed of distributing hydromorphone, an opiod pain medication. 

**1. See if a drug is mentioned among the "nodrugs" offenses and add it to the regular explression above. Regenerate the "nodrugs" data frame. What else needs to be added to our regular expression?**

In [ ]:
# Your code here



We can now create a column that has TRUE/FALSE whether a particular drug is mentioned. Here we add references to cocaine (is the word "cocaine" sufficient to find these offenses? Fix it if not.)

In [ ]:
newcommutations = mutate(newcommutations,cocaine=str_detect(tolower(offense),"cocaine"))
sample_n(newcommutations,5)

Next, we can make a breakdown of cocaine offenses by state. 

In [ ]:
table(newcommutations$state,newcommutations$cocaine)

**2. What do you observe? Are there any states that seem to have a different pattern than the others? If so, extract this state and tell me what offenses are frequent instead.**

In [ ]:
# Put your code here



In talking this over with Vice, they expressed interest in so-called ["851 enhancements"](https://www.avvo.com/legal-guides/ugc/21-u-s-c-851-federal-sentence-enhancements). Here is a description of 21 U.S.C. 851.

>A person who is charged in federal court may face an enhanced sentence if he or she has previously been convicted of a felony drug offense. 21 U.S.C. 851, also known as Section 851, is a subdivision of the Controlled Substances Act, which authorizes federal prosecutors to use a defendant's prior felony drug conviction to subject the defendant to an increased sentence in a current case. Federal prosecutors can use a prior drug conviction to enhance sentences in current drug, firearm or immigration cases. 

**3. Create a data frame called "eight51" that is a subset of "newcommutations" that contains all the offenses that have a reference to section 851. Use arrange() to sort it by the dates in "date_1" so that the 851 enhancements are ordered in time from oldest to newest.**

In [ ]:
# Put your code here



Assuming you created a data frame called eight51, you can use the following code to tell you about the years that people were sentenced with this enhancement. 

In [ ]:
table(year(eight51$date_1))

Now, tell me which states had this 851 indicator. 

**4. Use an expression like the one above to tell me what states had inmates receive the 851 enhancement.**

In [ ]:
# Put your code here



**Exploration**

In the above lessons, we've been creating new variables, creating subsets of data and examining relationships between different variables. There is a lot of work to do to get a sense of who is receiving commutations from Pres. Obama. Come up with a brief pitch around something you find in the data. This will require a little research about the "Clemency Initiative" as well as things like mandatory sentencing. Take this in whatever angle you like.

The pitch should be a few sentences and based in some data observation. The observation doesn't have to be rocket science... maybe baby steps to start.

Your work here!

